# Distribution of vegetation in a landscape of heterogeneous predation risk

In [ ]:
# importing libraries

import numpy as np
from matplotlib import pyplot as plt
import math
from itertools import permutations as perm
import time
from matplotlib import animation as animate

# Notebook parameters

plt.rcParams['figure.figsize'] = [8, 8]
plt.rcParams['font.size'] = 20

## Defining initialisation functions

In [ ]:
# Defining resource matrix

def veg(N):
    
    # Starting with resoource available everywhere
    
    V = np.full((N, N), 1)

    return V

# test plot

plt.matshow(veg(50))
plt.colorbar()

In [ ]:
# Creating list of fish in the landscape

def fish_vec(n, N):

    # n = initial number of fish
    # N = size of the landscape

    # Iniital position at the center

    x = int(N/2); y = int(N/2)

    # Data for a single fish

    fish = np.array([(x, y), 0]) # [(position), starvation time]

    fish_pop = np.full((n,2), fish)

    return fish_pop

# test

fish_vec(5, 50)

In [ ]:
# summarising fish density in the landscape

def fish_mat(fish_pop, N):

    # creating an empty matrix for fish

    F =  np.zeros((N, N))

    # Extracting fish positioins

    fish_pos = fish_pop[:,0]

    # Summing number of fish in each cell

    for index, value in np.ndenumerate(F):

        for f in fish_pos:

            if index == f:

                F[index] += 1

    return F

# test

fish_pop = fish_vec(20, 50)

plt.matshow(fish_mat(fish_pop, 50))
plt.colorbar()

In [ ]:
def risk_fun(dist, k =0.5, a = 50):
    
    return (1/(1 + math.exp(-(dist-a)*k)))

    #return 1/(a + math.exp((25 - dist)*k))

N = 50
center = N/2

ix = N/2
iy = list(range(N))

d = [0]*N

r = [0]*N
for i in range(N):

    d[i] = ((center - ix)**2 + (center - iy[i])**2)**0.5
    r[i] = risk_fun(d[i])




plt.plot(d, r)

In [ ]:
# Predation risk as a function of space

def risk(ix, iy, N = 50, k =0.5, a = 20):

    # Distance based change from the center

    center = N/2

    dist = ((center - ix)**2 + (center - iy)**2)**0.5

    alpha = (1/(1 + math.exp(-(dist-a)*k)))

    return alpha

# test

N = 50

x = list(range(N))
y = list(range(N))

index = 0 

alpha = [0]*50*50

for ix in x:

    for iy in y:

        alpha[index] =  risk(ix, iy, N)

        index += 1

alpha = np.array(alpha)

alpha = np.reshape(alpha, (50, 50))

plt.matshow(alpha)
plt.colorbar()

## Defining update functions

In [ ]:
# Periodic boundaries

def pbound(pt, N):

    if pt >= N-1:

        return(abs(pt - N))

    elif pt < 0:

        return(abs(pt + N))

    else:

        return(pt)

# test

pbound(51, 50)

In [ ]:
# spread and regeneration of vegetation

def veg_update(veg_mat, v=0.1):

    N = np.shape(veg_mat) # getting shape of vegetation matrix

    for index, value in np.ndenumerate(veg_mat):

        if value == 0:

            adj = [] # empty list of adjascent cells

            # getting list of adjascent cells

            for dx in [-1, 0, 1]:

                for dy  in [-1, 0, 1]:

                    x = index[0] + dx # horizontal

                    y = index[1] + dy # vertical

                    # Periodic boundaries

                    x = pbound(x, 50)
                    y = pbound(y, 50)

                    adj.append((x, y))

            # Spread of vegetation

            ## Getting values from adjascent cells

            adj_veg = []

            for cell in adj:

                adj_veg.append(veg_mat[cell])
            
            ## checking if neighbours are present

            if sum(adj_veg) > 0:

                # updating value 

                veg_mat[index] = int(np.random.choice(a = [0,1], p =  [1-v, v])) # prob of reproducing

    return(veg_mat)

# Test

V = veg(50)

for dx in range(10):

    for dy in range(10):

        V[dx, dy] = 0

V = veg_update(V)

plt.matshow(V)
plt.colorbar()


In [ ]:
# updating fish positions

def fish_mov(fish_pop, N = 50, k = 0.5, a = 20):

    index = 0

    for f in fish_pop:

        x = f[0][0]
        y = f[0][1]

        # Calculating risk

        xrisk = (risk(x+1, y, N, k = k, a = a) - risk(x-1, y, N, k = k, a = a))
        yrisk = (risk(x, y+1, N, k = k, a = a) - risk(x, y-1, N, k = k, a = a))

        # Calculating moves based on risk

        px =  0.5 - xrisk # Risk of moving right vs left
        py =  0.5 - yrisk # risk of moving up vs down

        # 2 - D risk - biased random walk

        dx = x + np.random.choice(a = [-1,1], p = [(1-px), px])
        dy = y + np.random.choice(a = [-1,1], p = [(1-py), py])

        # Periodic boundaries

        dx = pbound(dx, 50)

        dy = pbound(dy, 50)


        fish_pop[index][0] = (dx, dy)

        index += 1

    return fish_pop

# Test

fish_pop = fish_vec(20, 50)

fish_t1 = fish_mov(fish_pop)

plt.matshow(fish_mat(fish_t1, 50))
plt.colorbar()

In [ ]:
# Fish reproduction

def fish_rep(fish_pop, r = 0.01):

    for f in fish_pop:

        rep = np.random.choice(a = [0,1], p = [1-r, r]) # P(reproduce) 

        if rep == 1:

            f[1] = 0 #resetting starvation clock

            fish_pop = np.append(fish_pop, [f], axis = 0)
    
    return fish_pop


# Test

fish_pop = fish_vec(20, 50)

print(len(fish_rep(fish_pop)))

In [ ]:
# Fish starvation

def fish_starve(fish_pop):
    
    index = 0

    idx = []

    for f in fish_pop:

        # At t_S = 10, fish dies

        if f[1] == 10:

            idx.append(index)
        
        else:

            # Starvation clock increases with each time step

            fish_pop[index, 1] = fish_pop[index, 1] + 1

        index += 1
    
    # Killing fish

    fish_pop = np.delete(fish_pop, idx, axis = 0)

    return fish_pop

# Test

fish_pop = fish_vec(20, 50)

for t in range(12):

    fish_pop = fish_starve(fish_pop)

    print(len(fish_pop))


In [ ]:
# Fish feeding

def fish_feed(fish_pop, veg_mat, q = 5):

    index = 0

    for f in fish_pop:

        # Check if vegetation is present

        if veg_mat[f[0]] == 1:

            # Fish eats the vegetations 

            # Veg disappears

            veg_mat[f[0]] = 0

            # Fish starvation clock starts over

            fish_pop[index, 1] = fish_pop[index, 1] - q
        
        index += 1
    
    return veg_mat, fish_pop

# Test

fish_pop = fish_vec(20, 50)

veg_mat = veg(50)

veg_mat, fish_pop = fish_feed(fish_pop, veg_mat)

plt.matshow(veg_mat)
plt.colorbar()


## Simulating 

In [ ]:
# Defining simulation parameters

T_max = 500 # time for the simulation
N = 50 # length of side for N x N matrix
n = 20 # initial number of fish

In [ ]:
# Initialising the landscape

fish_pop = fish_vec(n, N) # fish population
veg_mat = veg(N) # resource matrix

In [ ]:
# Results

fish_abn = [0]*T_max # fish abundance
veg_abn = [0]*T_max # resource abundance

veg_res = np.zeros((N, N, T_max+2))
veg_res[:,:,0] = veg_mat

fish_res = np.zeros((N, N, T_max+2))
fish_res[:,:,0] = fish_mat(fish_pop, N)


In [ ]:
# Simulation

for t in range(T_max):

    # Updating resource values

    veg_mat = veg_update(veg_mat)

    # Fish feeding

    veg_mat, fish_pop = fish_feed(fish_pop, veg_mat)

    # Fish starvation and death

    fish_pop = fish_starve(fish_pop)

    # Fish reproduce

    fish_pop = fish_rep(fish_pop)

    # Fish movement

    fish_pop = fish_mov(fish_pop)

    # Logging fish abundance

    fish_abn[t] = len(fish_pop)

    fish_res[:,:,t+1] = fish_mat(fish_pop, N)

    # Logging resource

    veg_abn[t] = np.sum(veg_mat)

    veg_res[:,:,t+1] = veg_mat


Breaks for long simulation in either `fish_mov` or `fish_feed`

In [ ]:
# Animating fish movement in the landscape

## Scaling fish density

for t in range(T_max):

    fish_res[:,:,t] = fish_res[:,:,t]/np.sum(fish_res[:,:,t])

def lilly(slice):

    plt.clf()
    plt.imshow(fish_res[:,:,slice*10])
    plt.colorbar()
    plt.title(f"At time = {slice*10}")
    plt.clim(0, 0.1)

anim = animate.FuncAnimation(plt.figure(), lilly, range(T_max//10), interval = 500)

anim.save("sim1.gif")

In [ ]:
# Final vegetation distribution

plt.imshow(veg_mat)
plt.colorbar()

## Experiments

Varying risk gradient, vegetations input and fish reproductive rate

In [ ]:
# varying risk

N = 100
center = N/2

ix = N/2
iy = list(range(N))

d = [0]*N

r = [0]*N

k = [1, 0.5, 0.25, 0.125] # exponent

a = [5, 10, 20, 100] # half saturation distance

fig, axis = plt.subplots(4, 4, figsize = (16, 16))

plt.rcParams['font.size'] = '10'

for x in range(len(k)):

    for j in range(len(a)):

        for i in range(N):

            d[i] = ((center - ix)**2 + (center - iy[i])**2)**0.5
            
            r[i] = risk_fun(d[i], a = a[j], k = k[x])
        
        axis[x, j].plot(d, r)
        axis[x, j].set_title('a = %a; k = %a' % (a[j], k[x]))
        

In [ ]:
# varying distance and exponent of risk function

# Defining simulation parameters

T_max = 1000 # time for the simulation
N = 100 # length of side for N x N matrix
n = 20 # initial number of fish

# Results matrices

fish_abn = np.zeros((len(k), len(a), T_max))
veg_abn = np.zeros((len(k), len(a), T_max))

for i in range(len(k)):

    for j in range(len(a)):


        # Initialising the landscape

        fish_pop = fish_vec(n, N) # fish population
        veg_mat = veg(N) # resource matrix

        for t in range(T_max):

            # Updating resource values

            veg_mat = veg_update(veg_mat)

            # Fish feeding

            veg_mat, fish_pop = fish_feed(fish_pop, veg_mat)

            # Fish starvation and death

            fish_pop = fish_starve(fish_pop)

            # Fish reproduce

            fish_pop = fish_rep(fish_pop)

            # Fish movement

            fish_pop = fish_mov(fish_pop, N = 50, k = k[i], a = a[j])

            # Logging fish abundance

            fish_abn[i, j, t] = len(fish_pop)

            # Logging resource

            veg_abn[i, j, t] = np.sum(veg_mat)
    

In [ ]:
fig, axis = plt.subplots(4,4, figsize = (16, 16))

plt.rcParams['font.size'] = '10'

for i in range(len(k)):

    for j in range(len(a)):

        axis[i, j].plot(range(T_max), veg_abn[i,j,:])
        axis[i,j].set_title('a = %a; k = %a' % (a[j], k[i]))
        axis[i,j].set_ylim([np.min(veg_abn)-100, np.max(veg_abn)+100])

In [ ]:
fig, axis = plt.subplots(4,4, figsize = (32, 32))

plt.rcParams['font.size'] = '10'

for i in range(len(k)):

    for j in range(len(a)):

        axis[i, j].plot(range(T_max), fish_abn[i,j,:])
        axis[i,j].set_title('a = %a; k = %a' % (a[j], k[i]))
        axis[i,j].set_ylim([np.min(fish_abn)-10, np.max(fish_abn)+10])

In [ ]:
veg_eq = np.zeros((len(k), len(a)))
veg_var = np.zeros((len(k), len(a)))

for i in range(len(k)):

    for j in range(len(a)):

        veg_eq[i,j] = np.mean(veg_abn[i, j, (T_max-100):(T_max-1)])
        veg_var[i, j] = np.var(veg_abn[i, j, (T_max-100):(T_max-1)])
    
    plt.scatter(a, veg_eq[i,:], label = k[i])
    plt.plot(a, veg_eq[i,:])
    #plt.errorbar(a, veg_eq[i,:], label = k[i], yerr = veg_var[i,:], fmt='o')
    plt.legend()



In [ ]:
fish_eq = np.zeros((len(k), len(a)))
fish_var = np.zeros((len(k), len(a)))

for i in range(len(k)):

    for j in range(len(a)):

        fish_eq[i,j] = np.mean(fish_abn[i, j, (T_max-100):(T_max-1)])
        fish_var[i, j] = np.var(fish_abn[i, j, (T_max-100):(T_max-1)])
    
    plt.scatter(a, fish_eq[i,:], label = k[i])
    plt.plot(a, fish_eq[i,:])
    plt.legend()
